In [ ]:
from datetime import date
import pandas as pd
import re
import requests

## Pull data

In [ ]:
url = "https://www.lib.ncsu.edu/api/workshops/all"

In [ ]:
r = requests.get(url)
r

In [ ]:
data = r.json()

In [ ]:
data[0]

In [ ]:
df = pd.DataFrame(data)
df.head()

Strip white spaces from the `field_non_library_instructor` column. 

In [ ]:
df["field_non_library_instructor"] = df["field_non_library_instructor"].str.strip()
df.head()

Remove HTML tags from `body`.

In [ ]:
def strip_html_tags(text: str) -> str:
    tag = re.compile("<.*?>")
    return re.sub(tag, "", text)


In [ ]:
df["body"] = df["body"].apply(strip_html_tags)
df.head()

Write to csv, with date.

In [ ]:
df.to_csv(f"current-workshops-{date.today()}.csv")

## Explore text

TODO:
- Consistency across workshops
    - Length of description
    - Consistent keywords?
    - Does title reflect body paragraph? (check keywords)
        - Does body contain words from the title? Does title contain words from body?
    - Consistency of workshop descriptions across related titles
        - If we have multiple introductions to python, are the descriptions similar?
        - Similar across intro to R vs intro python? 
- Lexical variety?


In [ ]:
def count_words(text: str) -> int:
    return len([word for word in word_tokenize(text)
        if word not in [".", ",", "?", "'", "\"", ":", ";"]])

In [ ]:
count_words(df["body"][0])

In [ ]:
def show_words(text):
    words = text.replace("\n", " ")
    return words.split(" ")


In [ ]:
show_words(df["body"][0])

In [ ]:
from nltk import word_tokenize

In [ ]:
word_tokenize(df["body"][0])

In [ ]:
len([word for word in word_tokenize(df["body"][0]) 
    if word not in [".", ",", "?", "'", "\"", ":", ";"]])